In [1]:
import pandas as pd
from random import choices
import scipy.stats as st
import plotly.io
import plotly.graph_objects as go
from hypothesis_test import *

In [2]:
# Data preprocessing
df = pd.read_csv(r'Datasets/compas-scores.csv')
df = df[df['decile_score'] != -1]
df.shape

(11742, 47)

In [3]:
# Helper function for experiments
def group_decile_scores(score_list, num_groups):
    """
    This function groups scores together based on a given number of groups
    (num_groups). This would be the equivalent of sorting scores into
    different tiers (very low, low, etc.) where the number of tiers is
    num_groups. The function also returns a distribution of those tiers
    within the given score_list.
    """
    bound_increment = 10/num_groups
    group_list = []
    last_lower_bound = 0
    for i in range(num_groups):
        upper_bound = last_lower_bound + bound_increment
        group_list.append([last_lower_bound, upper_bound])
        last_lower_bound = upper_bound
    grouped_score_list = []
    for score in score_list:
        for group in group_list:
            if group[0] <= score and score <= group[1]:
                grouped_score_list.append(group_list.index(group))
                break
    grouped_score_distribution = [grouped_score_list.count(i)/len(grouped_score_list) for i in range(num_groups)]
    return (grouped_score_list, grouped_score_distribution)

In [4]:
# Setup and Experiments
AA_decile_scores = df[df['race'] == 'African-American']['decile_score'].tolist()
AA_decile_score_props = [AA_decile_scores.count(i)/len(AA_decile_scores) for i in range(1,11)]
possible_scores = list(range(1,11))
extrapolated_AA_scores = list(choices(possible_scores, AA_decile_score_props, k=100000))
white_decile_scores = df[df['race'] == 'Caucasian']['decile_score'].tolist()

alpha = 0.05
conf_interval_list = []
num_vals = list(range(2,11))

for val_count in num_vals:
    data = group_decile_scores(extrapolated_AA_scores, val_count)[0]
    val_list = list(range(val_count))
    hypothesis = group_decile_scores(white_decile_scores, val_count)[1]
    time = %timeit -n 1 -r 10 -o hypothesis_test_silent(data, val_list, alpha, hypothesis)
    units = 's'
    mean = time.average
    stdev = time.stdev
    if mean >= 60: # Convert from seconds to minutes if necessary
        mean /= 60
        stdev /= 60
        units = 'min'
    n = 100000 # Length of extrapolated dataset
    conf_interval = st.t.interval(0.95, df=n-1, loc=mean, scale=stdev)
    rounded_conf_interval = tuple(round(i, 2) for i in conf_interval)
    conf_interval_list.append(str(rounded_conf_interval) + ' ' + units)

fig = go.Figure(data=[go.Table(header=dict(values=['Num Labels', 'Run Time 95% CI']),
                cells=dict(values=[num_vals, conf_interval_list]))],
                layout=go.Layout(title=go.layout.Title(
                text="Hypothesis Test Run Time by Number of Labels")))
plotly.io.write_image(fig, r"Results/run_time_decile_score_results.pdf", format="pdf", engine="orca")

1.36 s ± 25.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.41 s ± 9.57 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
2.05 s ± 24.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.4 s ± 31.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
12.4 s ± 87.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
38.7 s ± 377 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
2min 5s ± 1.1 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
6min 40s ± 18.1 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
22min 49s ± 1min 35s per loop (mean ± std. dev. of 10 runs, 1 loop each)
